# Time of Use (ToU) Tariffs and Electricity Consumption
## Fourier Transform Version

How might total residential electrical usage change if consumers shift a portion of their consumption from peak to off peak hours?

In this approach, we note that usage data is "periodic," and use this observation as the basis for our shift by splitting the usage into frequencies and only shifting the usage with certain frequencies. 
The interested reader should consult our pdf writeup (hyperlink!!! Once we have our pdf, I'll upload it to the github and we can link to that) for details about the reasoning behind our assumptions, as we've given minimal explanation here in the interest of space.

This notebook contains sections that:

* Query and organize the usage data
* Define a weekhourly pricing scheme
* Perform a week-by-week shift
    - Define shifting functions using the Fourier transform
    - Conduct week-by-week shifts
        * Arrange user-defined testing usage data
        * Shift usage (assuming a single scheme for the whole year for simplicity) 
            * Our data is based on Vancouver so peak usage will be in the winter, therefore we use a winter ToU scheme for the whole year
* Display before and after and compare peak usage
    - Combine winter and summer shifted usage
    - Combine shifted usage with industrial and commercial usage




In [1]:
import getpass
import urllib.parse
import pandas as pd
import numpy as np
import datetime as dt
from plotly.subplots import make_subplots
import plotly.express as px
import scipy as scp
import sys
import datetime as dt

import mograph as mg
import data_management_functions as dmf

np.set_printoptions(threshold=sys.maxsize)
pd.set_option('display.max_columns', None)

**Connection**

Enter the EDM server address and the login credentials provided by Awesense. If you do not have the credentials, or have any trouble connecting, please contact api@awesense.com.
<span style='color:red'> **Please do NOT store the credentials in the notebook, nor share them with anyone.** </span>

In [2]:
edm_address = getpass.getpass(prompt='EDM server address: ')

print('\nEDM login information')
edm_name = getpass.getpass(prompt='Username: ')
edm_password = getpass.getpass(prompt='Password: ')
edm_password = urllib.parse.quote(edm_password)

%load_ext sql
%sql postgresql://$edm_name:$edm_password@$edm_address/edm
%config SqlMagic.displaycon = False
%config SqlMagic.feedback = False

# Delete the credential variables for security purposes.
del edm_name, edm_password

EDM server address: ········

EDM login information
Username: ········
Password: ········


In [3]:
# User input for the grid.
grid_id = input('Enter grid ID: ') # awefice

Enter grid ID: awefice


In [4]:
%%sql

SET timezone = 'America/Vancouver'

[]

In [5]:
%%sql result_meter <<

SELECT ge.grid_element_id AS meter_id,
    tdss.timestamp AT TIME ZONE 'America/Vancouver' AS timestamp,
    tdss.value AS "kWh",
    geds.type,
    ge.meta ->> 'type_of_consumer' as type_of_consumer
FROM grid_element AS ge
LEFT JOIN grid_element_data_source AS geds
    ON geds.grid_id = ge.grid_id
    AND geds.grid_element_id = ge.grid_element_id
JOIN ts_data_source_select(geds.grid_element_data_source_id, 'kWh', null) AS tdss
    ON TRUE
WHERE ge.type = 'Meter'
    AND ge.grid_id = '{grid_id}'
    AND geds.type = 'CONSUMER'
ORDER BY cast(split_part(ge.grid_element_id, '_', 2) AS int) asc, tdss.timestamp

Returning data to local variable result_meter


### Setting up dataframes, dropping duplicates, aggregating residential usage

We drop the November DST "extra hour" and aggregate by timestamp for all of the residential meters.

In [6]:
df_origin_raw = result_meter.DataFrame()

# Drop duplicate rows to handle Nov DST; 
df_origin = df_origin_raw.drop_duplicates(subset=['meter_id','timestamp'], keep='first')

# Aggregate usage from the df where we drop November duplicates
df_agg = dmf.ds_demand_cat(df_origin)

# Pick out just the residential usage
df_res = df_agg[['ds_kWh_res']].copy().rename(columns={"ds_kWh_res": "kWh"})

## Setting up the tariff function

We define the on, mid, and off peak hours, as well as on and off days and use them to define a numpy array of length 168 whose entries represent the cost of electricity at each hour of the week. In this case, we are using the Time of Use scheme from the [Ontario Energy Board](https://www.oeb.ca/consumer-information-and-protection/electricity-rates) which has different hours in the winter vs in the summer.

In [7]:
#Functions for the hourly price of electricity for a given week
def week_tariff_scheme(off_days, on_days, ondays_off, ondays_mid, ondays_peak, off_price, mid_price, peak_price):
    """
    Parameters:
        off_days: length 7 array indicating days with flat (lowest) tariff
        on_days: length 7 array indicating days with variable tariff
            needs to sum to the identity to make sense
        
        ondays_off: length 24 array indicating which hours of a day are low price when tariff is variable
        ondays_mid: length 24 array indicating which hours are middle price when tariff is variable
        ondays_peak: length 24 array indicating which hours are peak price when tariff is variable
            needs to sum to the identity to make sense
            
        off_price: float value of lowest tariff
        mid_price: float value of midddle tariff
        peak_price: float value of highest tariff
        
    Returns:
        The hourly price of electricity for that week as as an length 168 array 
    """
    off = np.longdouble(np.zeros(168))
    mid = np.longdouble(np.zeros(168))
    peak = np.longdouble(np.zeros(168))
    for d in range(7):
        for h in range(24):
            off[24*d+h] = off_days[d] + on_days[d]*ondays_off[h]
            mid[24*d+h] = on_days[d]*ondays_mid[h]
            peak[24*d+h] = on_days[d]*ondays_peak[h]
    return off_price*off + mid_price*mid + peak_price*peak

In [8]:
offdays = np.array([0,0,0,0,0,1,1]) # weekend
ondays = np.array([1,1,1,1,1,0,0]) # week

# winter hours
ondaysoffw = np.array([1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1])
ondayspeakw = np.array([0,0,0,0,0,0,0,1,1,1,1,0,0,0,0,0,0,1,1,0,0,0,0,0])
ondaysmidw = np.array([0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,0,0,0,0,0,0,0])

# winter pricing scheme
tariff_scheme_w = week_tariff_scheme(offdays, ondays, ondaysoffw, ondaysmidw, ondayspeakw, 8, 10, 12)

## A Week-by-Week Shift

#### Probability distributions based on frequency and tariff and spectral decomposition

In [9]:
def centered_mod(h):
    """
    h%168 but in (-84,84] 
    """
    x = h%168
    y = np.real((168/(2*np.pi*1j) )*(np.log(np.exp(2*np.pi*x*1j/168) ) ) )
    return np.int_(np.rint(y) )

def week_distance(h1, h2):
    """
    circle distance
    """
    return (84/np.pi)*np.arccos(np.cos(2*np.pi*(h1-h2)/168) )

def lin_weekdistance_effect_distr(freq, h):
    """
    tempering effect of weekdistance from h
    decays linearly
    returns 0 if freq%168 is zero
        (you shouldn't be able to move anything that you do 0 or 168 times per week)
    returns 1 if freq%168 is one
        (if you do it once a week you should be able to move it at any other time of the week)
    everything is fixed with np.sign because numpy is very inflexible with booleans
    """
    y=np.arange(168)
    
    abfr= np.abs(centered_mod(freq) )
    
    #to avoid 'division by zero' errors being thrown. not actually used in the math:
    fake_abfr = abfr + (1 - np.sign(np.abs(abfr) ) )*1e-64
  
    #zero at and only at abfr = 0 or 1:
    fake2_abfr = (np.sign(np.abs(abfr)))*(np.sign(np.abs(1-abfr)))

    X = np.sign(np.abs(fake2_abfr))*(168/fake_abfr -(week_distance(h,y)-1) ) + (1 -np.sign(np.abs(fake2_abfr)))*abfr

    return ( X + np.abs(X) )/2

def gaussian(x, mu, sigma):
    return np.exp(-0.5*np.power((x-mu)/sigma, 2))/(np.sqrt(2*np.pi)*sigma)

def gaussian_tempered_weekdistance_effect_distr(freq, h):
    x = np.arange(168)
    mu = h
    abfr = np.abs(centered_mod(freq) )
    fake_abfr = abfr + (1 - np.sign(np.abs(abfr) ) )*1e-64
    sigma = 168/(4*fake_abfr)
    return gaussian(x, mu, sigma)*lin_weekdistance_effect_distr(freq, h)

def gaussian_tempered_thrift_distribution(tariff, freq, h):
    """
    thrift distribution centered at h on circle
    """
    d168 = 1e-64 + gaussian_tempered_weekdistance_effect_distr(freq,h)/tariff
                    
    d168 *= ( 1/sum(d168) )
                    
    return d168

#### Functions for shifting and computing the shifting matrices

In [10]:
def gaussian_tempered_transpose_matrix_by_freq(tariff, freq):
    """
    given a tariff scheme and a frequency, 
        will output a shifting matrix based on the gaussian tempered thrift distribution 
        for that frequency and the tariff
    """
    transpose_matrix = np.zeros((168,168))
    for hour in range(168):
        transpose_matrix[hour] = gaussian_tempered_thrift_distribution(tariff, freq, hour)
    #def f(a,b):
    #    return gaussian_tempered_thrift_distribution(tariff, freq, a)[b] 
    #g = np.vectorize(f)
    #transpose_matrix = np.fromfunction(g, (168,168), dtype=int)
    return transpose_matrix

def shifted_basis_gaussian_tempered_matrix(tariff):
    """
    Precomputes the shift by applying it to the fourier basis for [0,168)
    Treats each frequency separately
    Outputs a 168x168 matrix we'll use to shift all the data we want for that specific tariff scheme
    """
    basis_t = scp.fft.ifft(np.eye(168))
    basis = np.transpose(basis_t)
    def shift(k):
        ThK = gaussian_tempered_transpose_matrix_by_freq(tariff, k)
        return np.matmul(basis[k],ThK)
    A = [shift(k) for k in range(168)]
    return np.array(A)

def precomputedspectralshift(to_shift, shiftedbasismatrix):
    """
    to_shift: a length 168 np.array with the kwh consumption for each hour of a week
    shiftedbasismatrix: pre-computed output of shifted_basis_matrix(tariff) for the tariff scheme we wish to apply
    
    Outputs the spectral shift of 'to_shift' by the tariff scheme 'tariff'
    """
    A = np.diag(scp.fft.fft(to_shift)[np.arange(168)])
    return (np.matmul(A,shiftedbasismatrix)).sum(axis=0)

In [11]:
SBgtM_w = shifted_basis_gaussian_tempered_matrix(tariff_scheme_w)

### Conducting a week-by-week shift

#### Preparing user-defined data for shifting

The user defines a time frame of interest and we show how the above shifting function adjusts (residential) user behavior assuming the winter shiftable percentages.

Ideally, the user will choose a time frame that begins on a Monday and ends on a Sunday (so it contains full weeks), but if not, the spare days on each end are omitted and stored so that if we choose we can perform a daily shift on them and add them back into the data we can. This could be implemented in future work; for now, we only consider full weeks.

In [12]:
# User input for the time frame.
user_start = input('Enter start date: ')
user_end = input('Enter end date (inclusive): ')

start_date = pd.to_datetime(user_start)
end_date = pd.to_datetime(user_end) + dt.timedelta(hours=23)

Enter start date: Jan 3, 2022
Enter end date (inclusive): Dec 25, 2022


In [13]:
# Get year, week, weekhour columns for just in the time frame
res_raw = dmf.timeframe_df(df_res, start_date, end_date)

# make a pivot table of the usage data (res_pivot), stripping off partial weeks at beginning and end
(spare_days, res_pivot) = dmf.pivot_strip_spare(res_raw)

# note: we could try to shift the spare days with a daily shift and add that back in. 
# I think it's not important, but I preserved them anyway

# make the pivot table into a numpy array that can be shifted
weekly_usage_array = res_pivot.to_numpy()

#### Shifting usage

In [14]:
# build a numpy array of shifted usage by looping over each week
weekly_shifted_array = np.zeros(weekly_usage_array.shape)

for week in range(weekly_shifted_array.shape[0]):
    weekly_shifted_array[week] = np.real(precomputedspectralshift(weekly_usage_array[week], SBgtM_w))
    
# insert the shifted array into a df that looks like the original pivot df
shifted_pivot = pd.DataFrame(weekly_shifted_array,
                             columns=list(res_pivot.columns),
                             index=res_pivot.index).reset_index()

#unpivot the df
shifted_dfmelt = pd.melt(shifted_pivot,
                                id_vars=['isoyear','week'],
                                value_vars=list(shifted_pivot.columns),
                               var_name='weekhour', value_name='ToU')\
                            .sort_values(by=['isoyear','week','weekhour'])

# merge with the unshifted data to retrieve timestamps and to have a comparison
orig_plus_shifted = pd.merge(res_raw, shifted_dfmelt, on=['isoyear','week','weekhour'])\
                        .set_index('timestamp')\
                        .drop(['isoyear','week','weekhour'], axis=1)

## Results

#### Recombining summer and winter and industrial/commercial

First, we recombine the shifted output for summer and winter, then merge with a data frame containing aggregated industrial and commercial usage. Finally, we add the industrial and commercial usage to both the unshifted and shifted residential usage.

In [15]:
# Retrieve just the commercial and industrial aggregated usage
df_agg_comm_ind = df_agg.drop(['ds_kWh_res'], axis=1)\
                            .rename(columns={"ds_kWh_comm": "comm",
                                             "ds_kWh_ind": "ind"
                                            }
                                   )\
                            .reset_index().copy()

# Merge with our shifted residential usage
df_agg_shift = pd.merge(orig_plus_shifted.reset_index(), df_agg_comm_ind, on=['timestamp'])\
                .rename(columns={'kWh': 'res_raw'})\
                .set_index('timestamp')


# Compute totals with the shifted vs unshifted data
df_agg_shift['no_shift']= df_agg_shift['res_raw']\
                            + df_agg_shift['comm']\
                            + df_agg_shift['ind']

df_agg_shift['shifted']= df_agg_shift['ToU']\
                            + df_agg_shift['comm']\
                            + df_agg_shift['ind']

### Usage before and after

#### Residential usage

In [22]:
mg.month_figure(orig_plus_shifted.reset_index(),\
               'Consumption Shifted Weekly',\
               ['kWh', 'ToU'],\
               ['Original Consumption', 'Shifted Consumption'],\
               t='timestamp', ytitle = 'Energy Consumption (kWh)')

In [23]:
# Daily maximum usage for the residential meters
mg.month_figure(dmf.daily_max(orig_plus_shifted).reset_index(),\
               'Daily Maximum Consumption Before and After Weekly Shift',\
               ['max_kWh', 'max_ToU'],\
               ['Original Max Consumption', 'Shifted Max Consumption'],\
               t='timestamp', ytitle = 'Energy Consumption (kWh)')

In [18]:
print("Pre shift peak consumption", orig_plus_shifted.kWh.max())
print("Post shift peak consumption", orig_plus_shifted.ToU.max())

Pre shift peak consumption 72.77423839657268
Post shift peak consumption 62.57187658698548


#### Total grid usage

In [19]:
# Plot the totals
mg.month_figure(df_agg_shift.reset_index(),\
               'Total Grid Electrical Consumption with Weekly Shift',\
               ['no_shift', 'shifted'],\
               ['No Residential Shift', 'Residential Shift'],\
               t='timestamp', ytitle = 'Energy Consumption (kWh)')

In [20]:
# Daily maximum usage for the total grid
mg.month_figure(dmf.daily_max(df_agg_shift).reset_index(),\
               'Daily Maximum Total Grid Consumption Before and After Weekly Shift',\
               ['max_no_shift', 'max_shifted'],
               ['Original Max Consumption', 'Shifted Max Consumption'],\
               t='timestamp', ytitle = 'Energy Consumption (kWh)')

In [21]:
print("Pre shift peak consumption", df_agg_shift.no_shift.max())
print("Post shift peak consumption", df_agg_shift.shifted.max())

Pre shift peak consumption 229.19793311518887
Post shift peak consumption 226.30660168182638
